In [1]:
# PART 1

# downloading packages
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# downloading clean dataset
# importing the dataset
clean_df1 = pd.read_csv("C:/Users/justi/Introduction-to-Python-S1-M1/Python-Introduction-Smoking-Prevalence/data/clean_df1.csv")
clean_df2 =pd.read_csv("C:/Users/justi/Introduction-to-Python-S1-M1/Python-Introduction-Smoking-Prevalence/data/clean_df2.csv")


In [ ]:
# PART 2

# regression of smoking prevalence on sex